In [6]:
import os
import sys
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

load_dotenv()  # Load API keys and other sensitive data from the .env file

True

In [7]:
api_key = os.getenv("APIKEY")

In [8]:
llm = ChatOpenAI(openai_api_key=api_key)  # Create the ChatGPT instance

In [9]:
llm.invoke("how can langsmith help with testing?")

AIMessage(content='Langsmith can help with testing in the following ways:\n\n1. Test automation: Langsmith can automate the testing process by writing test scripts in various programming languages to test software applications and systems.\n\n2. Test data generation: Langsmith can generate test data to simulate real-world scenarios and test the robustness of the software under different conditions.\n\n3. Test case generation: Langsmith can create test cases based on requirements and specifications to ensure comprehensive test coverage.\n\n4. Performance testing: Langsmith can conduct performance testing to evaluate the speed, responsiveness, and stability of the software under different load conditions.\n\n5. Regression testing: Langsmith can perform regression testing to ensure that new updates or changes to the software do not negatively impact existing functionality.\n\n6. Continuous integration and continuous testing: Langsmith can integrate testing into the software development pr

In [10]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

In [11]:
chain = prompt | llm 

In [12]:
chain.invoke({"input": "how can langsmith help with testing?"})

AIMessage(content='Langsmith can help with testing in a few ways:\n\n1. Test Automation: Langsmith can be used to automate testing tasks such as generating test data, running test scripts, and analyzing test results. This can help streamline the testing process and reduce manual effort.\n\n2. Language Support: Langsmith supports multiple programming languages, which can be helpful for testing different types of applications. Testers can write test scripts in their preferred language and use Langsmith to execute them.\n\n3. Code Quality Analysis: Langsmith can be used to perform code quality analysis, such as checking for code style violations, code complexity, and potential bugs. This can help improve the overall quality of the test code and make it more maintainable.\n\n4. Integration Testing: Langsmith can be integrated with other testing tools and frameworks to perform integration testing. This can help ensure that different components of the software work together correctly.\n\nOve

In [13]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

In [14]:
chain = prompt | llm | output_parser

In [15]:
chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a valuable tool for testing in several ways:\n\n1. Automated Testing: Langsmith can be used to generate test data automatically, which can save time and effort in setting up test environments and scenarios.\n\n2. Data Validation: Langsmith can help in validating data inputs and outputs, ensuring that the data being used in tests is accurate and consistent.\n\n3. Performance Testing: Langsmith can be used to generate large volumes of data to test the performance of systems under different loads and conditions.\n\n4. Stress Testing: Langsmith can be used to simulate stress conditions by generating extreme data values and testing how the system responds.\n\n5. Boundary Testing: Langsmith can help in testing boundary conditions by generating data at the limits of acceptable values and ensuring that the system behaves correctly.\n\nOverall, Langsmith can enhance the testing process by providing reliable and customizable data for a variety of testing scenarios.'

In [16]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com")

docs = loader.load()

In [18]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=api_key)

In [19]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

ImportError: Could not import faiss python package. Please install it with `pip install faiss-gpu` (for CUDA supported GPU) or `pip install faiss-cpu` (depending on Python version).

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain_core.documents import Document

document_chain.invoke({
    "input": "how can langsmith help with testing?",
    "context": [Document(page_content="langsmith can let you visualize test results")]
})

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

In [ ]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})